This notebook describes steps for installing the necessary software and genotyping TRs using using HipSTR, GangSTR, ExpansionHunter, and adVNTR. EnsembleTR is used to generate consensus TR genotypes from genotypes obtained from TR genotypers

# Obtain data

In [ ]:
# download toy dataset, about 2.09 GB
wget https://figshare.com/ndownloader/files/45851871

In [ ]:
# extract files from the archive 
tar -xvf example_data.tar.gz

In [13]:
# create output directory
mkdir -p output/{hipstr_output,advntr_output/advntr_dir,gangstr_output,eh_output,ensembletr_output}

tree output

output
├── advntr_output
│   └── advntr_dir
├── eh_output
├── ensembletr_output
├── gangstr_output
└── hipstr_output

6 directories, 0 files


# Installing tools

## bcftools and tabix

In [ ]:
# steps to install bcftools are described here, https://samtools.github.io/bcftools/howtos/install.html

git clone --recurse-submodules https://github.com/samtools/htslib.git
git clone https://github.com/samtools/bcftools.git
cd bcftools
make

## HipSTR

In [ ]:
# HipSTR

# install these dependencies if you do not have them by uncommenting the line below

# apt install g++ make zlib1g-dev libhts-dev libbz2-dev liblzma-dev

git clone https://github.com/gymrek-lab/HipSTR.git
cd HipSTR
make version && make

In [ ]:
./HipSTR --help
cd ..

## ExpansionHunter

In [ ]:
# ExpansionHunter
version=v5.0.0-linux_x86_64

wget https://github.com/Illumina/ExpansionHunter/releases/download/v5.0.0/ExpansionHunter-v5.0.0-linux_x86_64.tar.gz

tar -xzvf ExpansionHunter-v5.0.0-linux_x86_64.tar.gz


In [ ]:
# The ExpansionHunter executable can be found in 
ExpansionHunter-v5.0.0-linux_x86_64/bin/ExpansionHunter --help

## GangSTR

In [ ]:
 # GangSTR

mamba create -y --name gangstr
mamba activate gangstr
mamba config --add channels conda-forge
mamba config --add channels bioconda
mamba install -y -c conda-forge -c bioconda gangstr

In [ ]:
# this step maybe optional
# only run if you get this error
# GangSTR: error while loading shared libraries: libgsl.so.25: cannot open shared object file: No such file or directory
mamba install -y conda-forge::gsl

In [ ]:
GangSTR --help

mamba deactivate

## adVNTR

In [ ]:
# advntr
mamba create -y --name advntr
mamba activate advntr
mamba config --add channels conda-forge
mamba config --add channels bioconda
mamba install -y -c conda-forge -c bioconda advntr

In [ ]:
# this step is optional
# run if the step below returns
# WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'

mamba install -y mkl-service

In [ ]:
advntr 
mamba deactivate

## EnsembleTR

In [ ]:
# ensembleTR
git clone https://github.com/gymrek-lab/EnsembleTR.git
cd EnsembleTR
python3 setup.py install --user

In [ ]:
EnsembleTR --version
cd ..

## TRtools

In [ ]:
git clone https://github.com/gymrek-lab/TRTools.git
cd TRTools/
python3 -m pip install --upgrade pip
python3 -m pip install -e .
cd ..

## vcftools

In [ ]:
git clone https://github.com/vcftools/vcftools.git
cd vcftools
./autogen.sh
./configure
make
sudo make install

cd ..

In [6]:
# check software
./check_software.sh statSTR associaTR compareSTR dumpSTR mergeSTR prancSTR qcSTR simTR vcftools | grep -i 'not installed'

: 1

# Genotyping STRs and preprocessing VCF files

## HipSTR

In [41]:
# HipSTR

populations=("africa" "europe" "east_asia" "south_asia" "america")
reference_genome="references/reference_chroms.fa"

for population in "${populations[@]}"; do
    
    bam_files=$(cat str_resources/"${population}".txt | tr "\n" ",")

    HipSTR/HipSTR --bams $bam_files \
        --fasta $reference_genome \
        --regions str_resources/hipstr_reference.bed \
        --str-vcf output/hipstr_output/"${population}"_hipstr.vcf.gz \
        --log output/hipstr_output/"${population}"_strs.log \
        --viz-out output/hipstr_output/"${population}"_strs.viz.gz \
        --output-gls --output-pls --def-stutter-model
        
    zcat output/hipstr_output/"${population}"_hipstr.vcf.gz | vcf-sort | bgzip -c > output/hipstr_output/"${population}"_hipstr_sorted.vcf.gz
    tabix -p vcf output/hipstr_output/"${population}"_hipstr_sorted.vcf.gz
    
done

sort -k1,1d -k2,2n
sort -k1,1d -k2,2n
sort -k1,1d -k2,2n
sort -k1,1d -k2,2n
sort -k1,1d -k2,2n


### Understanding HipSTR output

In [16]:
vcftools --gzvcf output/hipstr_output/africa_hipstr.vcf.gz --snp TIMM10 --indv NA20357 --recode --recode-INFO-all --stdout | tail -n 2 | datamash transpose

#CHROM	chr11
POS	57528484
ID	TIMM10
REF	TATATATATATATATATATATATATATATATATA
ALT	TATATATATATATATATATATATA,TATATATATATATATATATATATATA,TATATATATATATATATATATATATACA,TATATATATATATATATATATATATATA,TATATATATATATATATATATATATATATA,TATATATATATATATATATATATATATATATA,TATATATATATATATATATATATATATATATATATATA
QUAL	.
FILTER	.
INFO	INFRAME_PGEOM=0.95;INFRAME_UP=0.05;INFRAME_DOWN=0.05;OUTFRAME_PGEOM=0.95;OUTFRAME_UP=0.01;OUTFRAME_DOWN=0.01;START=57528484;END=57528517;PERIOD=2;NSKIP=0;NFILT=0;BPDIFFS=-10,-8,-6,-6,-4,-2,4;DP=992;DSNP=0;DSTUTTER=22;DFLANKINDEL=13;AN=100;REFAC=5;AC=11,58,1,12,7,2,4
FORMAT	GT:GB:Q:PQ:DP:DSNP:DSTUTTER:DFLANKINDEL:PDP:PSNP:GLDIFF:AB:DAB:FS:ALLREADS:MALLREADS:GL:PL
NA20357	2|2:-8|-8:1.00:1.00:18:0:0:0:9.00|9.00:0|0:3.38:0:.:0:-8|12:-8|11:-109.14,-66.28,-62.67,-49.88,-49.66,-46.27,-73.62,-65.65,-51.31,-80.33,-65.93,-62.49,-49.64,-65.40,-62.32,-81.53,-66.01,-49.86,-72.64,-65.68,-77.93,-96.90,-66.26,-49.88,-73.57,-65.92,-81.29,-93.54,-112.73,-66.27,-49.87,-73.62,-65.93,-81.54,-97.15,-

### Visualising HipSTR alignments

In [19]:
# index the file with visualisation
tabix -p vcf output/hipstr_output/africa_strs.viz.gz

In [20]:
# visualising alignments at chr17:51831668 for Africa super population
## africa
HipSTR/VizAlnPdf output/hipstr_output/africa_strs.viz.gz chr17 51831668 NA20357 viz_NA20357 3 


In [44]:
convert -density 300 -quality 90 viz_NA20357.pdf -resize 25% NA20357.png
# display HG002330.png

![alignment](./NA20357.png)

## ExpansionHunter

In [39]:
reference_genome="references/reference_chroms.fa"
output_dir=output/eh_output
male_list=str_resources/bams_male.txt
female_list=str_resources/bams_female.txt
eh_reference_strs=str_resources/eh_reference.json

# males
for sample_bam in $(cat $male_list); do
        
    sample_id=$(basename $sample_bam | cut -f1 -d".")        

    ExpansionHunter-v5.0.0-linux_x86_64/bin/ExpansionHunter \
            --threads 32 \
            --reads $sample_bam \
            --reference $reference_genome \
            --variant-catalog $eh_reference_strs \
            --output-prefix $output_dir/$sample_id \
            --sex male \
            --log-level trace

    # eh vcf files do not have contigs listed in the header
    cat output/eh_output/"${sample_id}".vcf | vcf-sort | bgzip -c > output/eh_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/eh_output/"${sample_id}"_sorted.vcf.gz

done


# females
for sample_bam in $(cat $female_list); do
    
    sample_id=$(basename $sample_bam | cut -f1 -d".")        

    ExpansionHunter-v5.0.0-linux_x86_64/bin/ExpansionHunter \
            --threads 32 \
            --reads $sample_bam \
            --reference $reference_genome \
            --variant-catalog $eh_reference_strs \
            --output-prefix $output_dir/$sample_id \
            --sex female \
            --log-level trace
                
    # eh vcf files do not have contigs listed in the header
    cat output/eh_output/"${sample_id}".vcf | vcf-sort | bgzip -c > output/eh_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/eh_output/"${sample_id}"_sorted.vcf.gz
    
done

2024-04-22T19:43:16,[Starting ExpansionHunter v5.0.0]
2024-04-22T19:43:16,[Analyzing sample HG00108]
2024-04-22T19:43:16,[Initializing reference references/reference_chroms.fa]
2024-04-22T19:43:16,[Loading variant catalog from disk str_resources/eh_reference.json]
2024-04-22T19:43:16,[Running sample analysis in seeking mode]
2024-04-22T19:43:16,[Analyzing HTT]
2024-04-22T19:43:16,[Analyzing ESR2]
2024-04-22T19:43:16,[Collected 475 reads from (3):3073877-3075966]
2024-04-22T19:43:16,[Recovered 0 reads]
2024-04-22T19:43:16,[Analyzing CA10]
2024-04-22T19:43:16,[Analyzing NEXN]
2024-04-22T19:43:16,[Collected 183 reads from (0):77886913-77889010]
2024-04-22T19:43:16,[Recovered 0 reads]
2024-04-22T19:43:16,[Analyzing TIMM10]
2024-04-22T19:43:16,[Collected 282 reads from (10):57527484-57529517]
2024-04-22T19:43:16,[Could not recover the mate of A00217:72:HFKHYDSXX:1:2512:16387:18693/1]
2024-04-22T19:43:16,[Recovered 0 reads]
2024-04-22T19:43:16,[Collected 590 reads from (13):64252561-64254619

In [24]:
# merging sample VCFs into population VCFs

populations=("africa" "europe" "east_asia" "south_asia" "america")

for population in "${populations[@]}"; do
        
    # comma-seperated list of vcfs
    sample_vcfs=$(ls -1 output/eh_output/*_sorted.vcf.gz | \
    grep -f str_resources/"${population}"_ids.csv | tr "\n" "," | sed 's/,$//')
    
    mergeSTR \
        --vcfs $sample_vcfs \
        --vcftype 'eh' \
        --out output/eh_output/"${population}"_eh
        
    cat output/eh_output/"${population}"_eh.vcf | vcf-sort | bgzip -c output/eh_output/"${population}"_eh.vcf.gz
    tabix -p vcf output/eh_output/"${population}"_eh.vcf.gz
    
    # filtering with dumpSTR
#     dumpSTR --vcf output/eh_output/"${population}"_eh.vcf \
#         --out output/eh_output/"${population}"_eh_filt \
#         --vcftype 'eh' 
        
#     bgzip output/eh_output/"${population}"_eh_filt.vcf
#     tabix -p vcf output/eh_output/"${population}"_eh_filt.vcf.gz

done


## GangSTR

In [25]:
mamba activate gangstr

reference_genome="references/reference_chroms.fa"
output_dir=output/gangstr_output
male_list=str_resources/bams_male.txt
female_list=str_resources/bams_female.txt
gangstr_reference_strs=str_resources/gangstr_reference.bed

# male samples
for sample_bam in $(cat $male_list); do

    sample_id=$(basename $sample_bam | cut -f1 -d".")

    GangSTR --bam $sample_bam \
            --bam-samps $sample_id \
            --regions $gangstr_reference_strs \
            --samp-sex M \
            --ref $reference_genome \
            --out $output_dir/$sample_id \
            --insertmean 1 \
            --insertsdev 1 \
            --minmatch 1 \
            --min-sample-reads 1

    cat output/gangstr_output/"${sample_id}".vcf | vcf-sort | bgzip -c > output/gangstr_output/"${sample_id}"_sorted.vcf.gz

done


#female samples
for sample_bam in $(cat $female_list); do

    sample_id=$(basename $sample_bam | cut -f1 -d".")

    GangSTR --bam $sample_bam \
            --bam-samps $sample_id \
            --regions $gangstr_reference_strs \
            --samp-sex F \
            --ref $reference_genome \
            --out $output_dir/$sample_id \
            --insertmean 1 \
            --insertsdev 1 \
            --minmatch 1 \
            --min-sample-reads 1

    cat output/gangstr_output/"${sample_id}".vcf | vcf-sort | bgzip -c > output/gangstr_output/"${sample_id}"_sorted.vcf.gz
            

done

mamba deactivate

[GangSTR-2.5.0] ProgressMeter: Loading read group  HG00108 for file 1000_genomes/HG00108.bam
[GangSTR-2.5.0] ProgressMeter: Processing chr1:77887913
[GangSTR-2.5.0] ProgressMeter: 	Genotyper Results:  1, 1	likelihood = -25
[GangSTR-2.5.0] ProgressMeter: Processing chr4:3074877
[GangSTR-2.5.0] ProgressMeter: 	Genotyper Results:  17, 19	likelihood = 411.691
[GangSTR-2.5.0] ProgressMeter: Processing chr11:57528484
[GangSTR-2.5.0] ProgressMeter: 	Genotyper Results:  13, 17	likelihood = 66.5938
[GangSTR-2.5.0] ProgressMeter: Processing chr14:64253561
[GangSTR-2.5.0] ProgressMeter: 	Genotyper Results:  31, 32	likelihood = 109.884
[GangSTR-2.5.0] ProgressMeter: Processing chr17:51831668
[GangSTR-2.5.0] ProgressMeter: 	Genotyper Results:  10, 21	likelihood = 224.468
[GangSTR-2.5.0] ProgressMeter: Processing chr21:43776443
[GangSTR-2.5.0] ProgressMeter: 	Genotyper Results:  1, 1	likelihood = -25
Writing to /tmp/bcftools.fqqbrR
Merging 1 temporary files
Cleaning
Done
[GangSTR-2.5.0] ProgressMete

In [27]:
# merging sample VCFs to population VCF
populations=("africa" "europe" "east_asia" "south_asia" "america")

for population in "${populations[@]}"; do
        
    sample_vcfs=$(ls -1 output/gangstr_output/*_sorted.vcf.gz | \
    grep -f str_resources/"${population}"_ids.csv | tr "\n" "," | sed 's/,$//')
    
    mergeSTR \
        --vcfs $sample_vcfs \
        --vcftype 'gangstr' \
        --out output/gangstr_output/"${population}"_gangstr
        
        
    cat output/gangstr_output/"${population}"_gangstr.vcf | vcf-sort | bgzip -c > output/gangstr_output/"${population}"_gangstr.vcf.gz
    tabix -p vcf output/gangstr_output/"${population}"_gangstr.vcf.gz
    
done

## adVNTR

In [43]:
# view VNTR ID of CSTB gene

mamba activate advntr

advntr viewmodel --gene CSTB --pattern GCGCGGGGCGGG --models str_resources/vntr_data/hg19_selected_VNTRs_Illumina.db

Using Theano backend.
VNTR ID	| Chr	| Gene	| Start Position | Pattern
--------------------------------------------------
301645	| chr21	|CSTB	| 45196323	 | GCGCGGGGCGGG


In [28]:
advntr_model=str_resources/vntr_data/hg19_selected_VNTRs_Illumina.db
output_dir=output/advntr_output
advntr_dir=output/advntr_output/advntr_dir
bam_list=str_resources/bam_list.txt


# main script
for sample_bam in $(cat $bam_list); do

    sample_id=$(basename $sample_bam | cut -f1 -d".")

    advntr genotype \
            --vntr_id 301645 \
            --alignment_file $sample_bam \
            --outfmt vcf \
            --outfile $output_dir/"${sample_id}".vcf \
            --models $advntr_model \
            --working_directory $advntr_dir
    
    cat $output_dir/"${sample_id}".vcf | vcf-sort | bgzip -c > $output_dir/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/advntr_output/"${sample_id}".vcf.gz


done

mamba deactivate

Using Theano backend.
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 284 reads
Writing to /tmp/bcftools.JGxEcL
Merging 1 temporary files
Cleaning
Done
Using Theano backend.
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 278 reads
Writing to /tmp/bcftools.2OHy3d
Merging 1 temporary files
Cleaning
Done
Using Theano backend.
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 281 reads
Writing to /tmp/bcftools.7L7phe
Merging 1 temporary files
Cleaning
Done
Using Theano backend.
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 206 reads
Writing to /tmp/bcftools.yX4jRp
Merging 1 temporary files
Cleaning
Done
Using Theano backend.
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 233 reads
Writing to /tmp/bcftools.kLBGfi
Merging 1 temporary files
Cleaning
Done
Using Theano backend.
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 207 read

In [37]:
populations=("africa" "europe" "east_asia" "south_asia" "america")

for population in "${populations[@]}"; do
        
    # comma-seperated list of vcfs
    sample_vcfs=$(ls -1 output/advntr_output/*_sorted.vcf.gz | \
    grep -f str_resources/"${population}"_ids.csv | tr "\n" "," | sed 's/,$//')
    
    mergeSTR \
        --vcfs $sample_vcfs \
        --vcftype 'advntr' \
        --out output/advntr_output/"${population}"_advntr

    cat output/advntr_output/"${population}"_advntr.vcf | vcf-sort | bgzip -c > output/advntr_output/"${population}"_advntr.vcf.gz
    tabix -p vcf output/advntr_output/"${population}"_advntr.vcf.gz


done

sort -k1,1d -k2,2n
sort -k1,1d -k2,2n
sort -k1,1d -k2,2n
sort -k1,1d -k2,2n
sort -k1,1d -k2,2n


## Merging population VCFs from each tools

In [36]:
# gangstr
mergeSTR \
    --vcfs output/gangstr_output/africa_gangstr.vcf.gz,output/gangstr_output/east_asia_gangstr.vcf.gz,output/gangstr_output/south_asia_gangstr.vcf.gz,output/gangstr_output/america_gangstr.vcf.gz,output/gangstr_output/europe_gangstr.vcf.gz \
    --vcftype 'gangstr' \
    --out output/ensembletr_output/gangstr_strs

cat output/ensembletr_output/gangstr_strs.vcf | vcf-sort | bgzip -c > output/ensembletr_output/gangstr_strs.vcf.gz
tabix -p vcf output/ensembletr_output/gangstr_strs.vcf.gz

# expansionhunter
mergeSTR \
    --vcfs output/eh_output/africa_eh.vcf.gz,output/eh_output/east_asia_eh.vcf.gz,output/eh_output/south_asia_eh.vcf.gz,output/eh_output/america_eh.vcf.gz,output/eh_output/europe_eh.vcf.gz \
    --vcftype 'eh' \
    --out output/ensembletr_output/eh_strs

cat output/ensembletr_output/eh_strs.vcf | vcf-sort | bgzip -c > output/ensembletr_output/eh_strs.vcf.gz
tabix -p vcf output/ensembletr_output/eh_strs.vcf.gz

#advntr
mergeSTR \
    --vcfs output/advntr_output/africa_advntr.vcf.gz,output/advntr_output/east_asia_advntr.vcf.gz,output/advntr_output/south_asia_advntr.vcf.gz,output/advntr_output/america_advntr.vcf.gz,output/advntr_output/europe_advntr.vcf.gz \
    --vcftype 'advntr' \
    --out output/ensembletr_output/advntr_strs 
        
cat output/ensembletr_output/advntr_strs.vcf | vcf-sort | bgzip -c > output/ensembletr_output/advntr_strs.vcf.gz
tabix -p vcf output/ensembletr_output/advntr_strs.vcf.gz

# hipstr
mergeSTR \
    --vcfs output/hipstr_output/africa_hipstr_sorted.vcf.gz,output/hipstr_output/east_asia_hipstr_sorted.vcf.gz,output/hipstr_output/south_asia_hipstr_sorted.vcf.gz,output/hipstr_output/america_hipstr_sorted.vcf.gz,output/hipstr_output/europe_hipstr_sorted.vcf.gz \
    --vcftype 'hipstr' \
    --out output/ensembletr_output/hipstr_strs

cat output/ensembletr_output/hipstr_strs.vcf | vcf-sort | bgzip -c > output/ensembletr_output/hipstr_strs.vcf.gz
tabix -p vcf output/ensembletr_output/hipstr_strs.vcf.gz

sort -k1,1d -k2,2n
sort -k1,1d -k2,2n
sort -k1,1d -k2,2n
sort -k1,1d -k2,2n


## EnsembleTR consensus genotyping

In [35]:
reference_genome="references/reference_chroms.fa"

EnsembleTR \
    --ref $reference_genome \
    --vcfs output/ensembletr_output/advntr_strs.vcf.gz,output/ensembletr_output/eh_strs.vcf.gz,output/ensembletr_output/gangstr_strs.vcf.gz,output/ensembletr_output/hipstr_strs.vcf.gz \
    --out output/ensembletr_output/ensembletr.vcf
    
cat output/ensembletr_output/ensembletr.vcf | vcf-sort | bgzip -c > output/ensembletr_output/ensembletr.vcf.gz
tabix -p vcf output/ensembletr_output/ensembletr.vcf.gz

[W::vcf_parse_format_fill5] Extreme FORMAT/DAB value encountered and set to missing at chr4:3074877
Analysing record cluster ranged in chr21:45196324-45196359.
Analysing record cluster ranged in chr1:77887914-77888010.
Analysing record cluster ranged in chr1:77887913-77888010.
Analysing record cluster ranged in chr4:3074878-3074933.
Analysing record cluster ranged in chr4:3074877-3074933.
Analysing record cluster ranged in chr4:3074877-3074968.
Analysing record cluster ranged in chr4:3074940-3074966.
Analysing record cluster ranged in chr11:57528485-57528517.
Analysing record cluster ranged in chr11:57528484-57528517.
Analysing record cluster ranged in chr11:57528484-57528517.
Analysing record cluster ranged in chr14:64253562-64253619.
Analysing record cluster ranged in chr14:64253561-64253619.
Analysing record cluster ranged in chr14:64253561-64253619.
Analysing record cluster ranged in chr17:51831669-51831730.
Analysing record cluster ranged in chr17:51831668-51831730.
Analysing reco